# Bedrock with LangChain using a Prompt that includes Context

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Setup

In [1]:
import json
import os
import sys
import boto3

# Add any necessary paths for custom modules
module_path = ".."
sys.path.append(os.path.abspath(module_path))

from labutils import bedrock, print_ww

# Uncomment and edit the following lines for your AWS setup
# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g., "us-west-2"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [2]:
%pip install pydantic==1.10.7
%pip install langchain-community==0.2.11
%pip install sagemaker-jupyterlab-extension-common
%pip install anthropic
%pip install langchain-aws==0.1.15  # or the compatible version

  Using cached pydantic-1.10.7-py3-none-any.whl.metadata (145 kB)
Using cached pydantic-1.10.7-py3-none-any.whl (157 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.7
    Uninstalling pydantic-2.11.7:
      Successfully uninstalled pydantic-2.11.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-ai 2.31.6 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
amazon-sagemaker-jupyter-ai-q-developer 1.2.7 requires pydantic<3,>=2.10.0, but you have pydantic 1.10.7 which is incompatible.
amzn-sagemaker-aiops-jupyterlab-extension 1.0.4 requires pydantic<3,>=1.10.17, but you have pydantic 1.10.7 which is incompatible.
jupyter-ai 2.31.6 requires pydantic<3,>=2.10.0, but you have pydantic 1.10.7 which is incompatible.
jupyter-ai-magics 2.31.6 requires langchain<0.4.0,>=0.3.0, but you have langchain 0.2.17 whi

## Reference the Bedrock LLM

In [3]:
from langchain_aws import BedrockLLM

inference_modifier = {
    'max_tokens_to_sample': 4096, 
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"]
}

textgen_llm = BedrockLLM(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock, 
    model_kwargs=inference_modifier 
)


## Create a LangChain custom prompt template

In [4]:
from langchain.prompts import PromptTemplate

# Create a prompt template with multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""
    Human: Create an apology email from the Service Manager {customerServiceManager} to {customerName} in response to the following feedback that was received from the customer: 
    <customer_feedback>
    {feedbackFromCustomer}
    </customer_feedback>

    Assistant:"""
)

# Fill in the template with specific values
prompt = multi_var_prompt.format(
    customerServiceManager="Swami", 
    customerName="John Doe", 
    feedbackFromCustomer="""Hello Swami,
         I am very disappointed with the recent experience I had when I called your customer support.
         I was expecting an immediate call back but it took three days for us to get a call back.
         The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
         We are very unhappy with the response provided and may consider taking our business elsewhere."""
)


### Print the number of tokens in the prompt (for interest/debug)

In [6]:
%pip install tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.9 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import tiktoken

# Initialize the tokenizer for the 'claude-v2' model
enc = tiktoken.get_encoding("cl100k_base")  # Anthropic Claude uses this encoding

# Count tokens in the prompt
num_tokens = len(enc.encode(prompt))
print(f"Our prompt has {num_tokens} tokens")


Our prompt has 125 tokens


In [9]:
print(prompt)


    Human: Create an apology email from the Service Manager Swami to John Doe in response to the following feedback that was received from the customer: 
    <customer_feedback>
    Hello Swami,
         I am very disappointed with the recent experience I had when I called your customer support.
         I was expecting an immediate call back but it took three days for us to get a call back.
         The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
         We are very unhappy with the response provided and may consider taking our business elsewhere.
    </customer_feedback>

    Assistant:


## Invoke the model and print the response

In [10]:
response = textgen_llm.invoke(prompt)

email = response[response.index('\n')+1:]
print_ww(email)



I want to sincerely apologize for the poor service you recently received from our customer support
team. It is unacceptable that it took multiple days and incorrect suggestions before we were able to
resolve your issue.

At [Company], our top priority is providing excellent customer service. I am very disappointed to
hear that we failed to meet your expectations and live up to our standards. You deserve much better.

I assure you that we are taking steps to prevent this from happening again. I have met with our
customer support team to review this incident and identify areas for improvement in our processes.
We will be implementing additional training and quality control measures to ensure we respond
promptly and accurately to all customer inquiries going forward.

I understand that this experience has caused frustration for you. Please accept my deepest
apologies. I hope you will give us another chance to regain your confidence and provide the
excellent service you expect and deserve